In [32]:
import gym
import numpy as np

In [33]:
env = gym.make("Taxi-v2")
print("initial observation code:", env.reset())
env.render()
n_states = env.observation_space.n
n_actions = env.action_space.n

initial observation code: 184
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [34]:
action_to_i = {
    'south': 0,
    'north': 1,
    'east': 2,
    'west': 3,
    'pickup': 4,
    'dropoff': 5,
}

In [35]:
env.step(action_to_i['south'])
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)


In [36]:
def sample_reward(env, policy, t_max=52):
    """
    Interact with an environment, return sum of all rewards.
    If game doesn't end on t_max (e.g. agent walks into a wall), 
    force end the game and return whatever reward you got so far.
    Tip: see signature of env.step(...) method above.
    """
    s = env.reset()
    total_reward = 0
    
    for _ in range(t_max):
        s, reward, is_done, _ = env.step(policy[s])
        total_reward += reward
        if reward == 20:
            break
    return total_reward

## Genetic algorithm

In [37]:
def crossover(policy1, policy2, p=0.5):
    """
    for each state, with probability p take action from policy1, else policy2
    """
    choice = np.random.binomial(1, [p], size=len(policy1))
    return np.array([policy1[i] if choice[i] else policy2[i] for i in range(len(policy1))])

In [38]:
def get_random_policy():
    return np.random.choice(n_actions, size=n_states)

In [39]:
def mutation(policy, p=0.1):
    """
    for each state, with probability p replace action with random action
    Tip: mutation can be written as crossover with random policy
    """
    return crossover(get_random_policy(), policy, p)  
    

In [40]:
np.random.seed(1234)
policies = [crossover(get_random_policy(), get_random_policy()) 
            for i in range(10**4)]

assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should be n_actions-1'

assert any([np.mean(crossover(np.zeros(n_states), np.ones(n_states))) not in (0, 1)
               for _ in range(100)]), "Make sure your crossover changes each action independently"
print("Seems fine!")

Seems fine!


In [41]:
n_epochs = 100  # how many cycles to make
pool_size = 20  # how many policies to maintain
n_crossovers = 50  # how many crossovers to make on each step
n_mutations = 50  # how many mutations to make on each tick

In [42]:
def evaluate(policy, n_times=150):
    """Run several evaluations and average the score the policy gets."""
    rewards = [sample_reward(env, policy) for _ in range(n_times)]
    return float(np.mean(rewards))
        

In [43]:
print("initializing...")
pool = [get_random_policy() for _ in range(pool_size)]
pool_scores = [evaluate(policy) for policy in pool]


initializing...


In [44]:
assert type(pool) == type(pool_scores) == list
assert len(pool) == len(pool_scores) == pool_size
assert all([type(score) in (float, int) for score in pool_scores])


In [45]:
#main loop
for epoch in range(n_epochs):
    print("Epoch %s:"%epoch)
    
    crossovered = [crossover(pool[p1], pool[p2]) 
                   for p1, p2 in zip(np.random.choice(len(pool), size=n_crossovers), 
                                     np.random.choice(len(pool), size=n_crossovers))]
    mutated = [mutation(pool[p]) for p in np.random.choice(len(pool), size=n_mutations)]
    
    assert type(crossovered) == type(mutated) == list
    
    #add new policies to the pool
    pool.extend(crossovered)
    pool.extend(mutated)
    pool_scores = [evaluate(p) for p in pool]
    
    #select pool_size best policies
    selected_indices = np.argsort(pool_scores)[-pool_size:]
    pool = [pool[i] for i in selected_indices]
    pool_scores = [pool_scores[i] for i in selected_indices]

    #print the best policy so far (last in ascending score order)
    print("best score:", pool_scores[-1])

Epoch 0:
best score: -248.62
Epoch 1:
best score: -215.74
Epoch 2:
best score: -235.0
Epoch 3:
best score: -206.02
Epoch 4:
best score: -200.14
Epoch 5:
best score: -188.08
Epoch 6:
best score: -188.14
Epoch 7:
best score: -188.2
Epoch 8:
best score: -163.36
Epoch 9:
best score: -160.18
Epoch 10:
best score: -163.48
Epoch 11:
best score: -144.4
Epoch 12:
best score: -141.7
Epoch 13:
best score: -131.74
Epoch 14:
best score: -132.34
Epoch 15:
best score: -110.98
Epoch 16:
best score: -114.16
Epoch 17:
best score: -111.16
Epoch 18:
best score: -101.68
Epoch 19:
best score: -122.98
Epoch 20:
best score: -107.74
Epoch 21:
best score: -95.38
Epoch 22:
best score: -95.5
Epoch 23:
best score: -92.5
Epoch 24:
best score: -92.2
Epoch 25:
best score: -95.68
Epoch 26:
best score: -89.32
Epoch 27:
best score: -85.84
Epoch 28:
best score: -85.96
Epoch 29:
best score: -92.5
Epoch 30:
best score: -83.08
Epoch 31:
best score: -80.02
Epoch 32:
best score: -79.9
Epoch 33:
best score: -79.96
Epoch 34:
be

Вроде и сходится, но не до конца. Видимо, здесь лучше использовать Qlearning.